## Conociendo un poco nuestro DATASET

#### Las limpiezas que se deben realizar son las siguientes:

1- `belongs_to_collection`, `production_companies`, `genres`, `spoken_languages`: los datos estan anidados
    Se debe buscar la forma de desanidarlos, o bien, buscar la forma de que los datos se puedan consultar sin desanidarlos.

2- `revenue`, `budget`: poseen datos nulos.
    Se deben rellenar estos datos con *0*.

3- `release_date`: posee datos nulos.
    Se deben eliminar estos datos.

4- `Campos con fecha`: no poseen el formato correcto.
    Se debe transformar cada uno de estos campos al formato *AAAA-mm-dd*.
    Se debe crear una nueva columna `release_year` donde se extraerá el año de la fecha de estreno.

5- `revenue`, `budget`: a partir de estos campos se debe crear una nueva columna `return`, retorno de inversión.
    Esta columna sera el resultado de `revenue`/`budget` y cuando no hay datos disponibles debera retornar *0*.
    
6- `video`, `imdb_id`, `adult`, `original_title`, `poster_path`, `homepage`: no utilizaremos estos campos.
    Se deberan eliminar.

In [45]:
# importamos las librerias necesarias para limpieza y transformacion:
import pandas as pd
import numpy as np

### Movies

### Pequeño análisis exploratorio

In [46]:
# creamos nuestro dataframe
mv = pd.read_csv('movies_dataset.csv', delimiter=',')
mv.tail(4)

C:\Users\Martin\AppData\Local\Temp\ipykernel_6664\780545667.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  mv = pd.read_csv('movies_dataset.csv', delimiter=',')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [4]:
# vemos el tamaño de nuestro dataframe
mv.shape

(45466, 24)

In [5]:
mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
# vemos sus columnas
mv.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [7]:
mv.id.duplicated().value_counts() # tenemos 30 registros duplicados, pero la consigna no nos dice que debemos borrarlos.

False    45436
True        30
Name: id, dtype: int64

---------------------
### Se podria comenzar por el item 6 para tener un numero de columnas mas reducido y un mejor uso de los recursos a la hora de realizar las trasnformaciones. No es lo mismo transformar un df con 24 columnas que un df con 18 columnas.
------

### 6- `video`, `imdb_id`, `adult`, `original_title`, `poster_path`, `homepage`: no utilizaremos estos campos.
    Se deberan eliminar.

In [19]:
mv.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [9]:
# verificamos que los campos se hayan borrado
mv.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

-----
### Con un razonamiento parecido al anterior, seria recomendable seguir por el punto 3. En este punto, se deben eliminar registros en los que 'release_date' posea datos nulos. Con la eliminacion de estos registros estariamos optimizando la utilizacion de los recursos de procesamiento. No es lo mismo un df de 44000 registros que un df de 40000.
-----

### 3- `release_date`: posee datos nulos.
    Se deben eliminar estos datos.

In [10]:
# inicialmente nuestro dataframe mv posee 45466 registros
mv.shape

(45466, 18)

In [20]:
mv.dropna(subset=['release_date'], inplace=True) # con el parametro subset defino en que columna voy a buscar los valores NaN para dropearlos

In [12]:
# una vez dropeados estos registros tendremos:
print(mv.shape)
print(f'Se han eliminado {45466-45379} registros')

(45379, 18)
Se han eliminado 87 registros


### 2- `revenue`, `budget`: poseen datos nulos.
    Se deben rellenar estos datos con *0*.

In [13]:
mv['revenue'].info()
# vemos que existen 45376 datos no nulos y 3 nulos

<class 'pandas.core.series.Series'>
Int64Index: 45379 entries, 0 to 45465
Series name: revenue
Non-Null Count  Dtype  
--------------  -----  
45376 non-null  float64
dtypes: float64(1)
memory usage: 709.0 KB


In [21]:
mv['revenue'].fillna(0,inplace=True) # este metodo llena todos los valores NaN de 'revenue' con el valor que nosotros le especifiquemos

In [15]:
mv['revenue'].info()
# vemos que ahora los datos no nulos son 45379, el total del df

<class 'pandas.core.series.Series'>
Int64Index: 45379 entries, 0 to 45465
Series name: revenue
Non-Null Count  Dtype  
--------------  -----  
45379 non-null  float64
dtypes: float64(1)
memory usage: 709.0 KB


Hacemos lo mismo para 'budget'

In [16]:
mv['budget'].astype('float') # vemos que cuando queremos pasar todo a float nos tira un error de tipo
# ValueError: could not convert string to float: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'


ValueError: could not convert string to float: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

In [25]:
# hacemos una copia que nos servira para luego verificar cuantos registros se transformaron
mv_budget_copia = mv['budget']
mv_budget_copia

0        30000000.0
1        65000000.0
2               0.0
3        16000000.0
4               0.0
            ...    
45460           0.0
45462           0.0
45463           0.0
45464           0.0
45465           0.0
Name: budget, Length: 45379, dtype: float64

In [ ]:
# seria el mismo error que tendriamos si quisieramos hacer:
a = "asasdasffasf"
float(a)

In [22]:
# definiremos una funcion para poder manipular este error:
def transf(x):
    try: 
        x = float(x) # intento transformar mi variable a float
    except ValueError: # de no poderse por un ValueError, voy a reemplazarlo por 0.0 (float)
        x = 0.0
    return x

In [24]:
# definiremos otra funcion para limpiar mv_budget_copia para luego saber que transformaciones hubo en nuestra columna budget
def transf_b(x):
    try: 
        x = float(x) # intento transformar mi variable a float
    except ValueError: # de no poderse por un ValueError, voy a reemplazarlo por algo que me indique que aqui hubo una trnasformacion
        x = 'transformado'
    return x

In [ ]:
var_de_prueba = "asdasd"
transf(var_de_prueba)

In [26]:
mv['budget'] = mv['budget'].apply(transf) # aplico transformacion a la columna 'budget' original
mv_budget_copia = mv_budget_copia.apply(transf_b) # aplico transformacion a la columna 'budget' para verificacion de cambios


In [27]:
df_cambios = mv_budget_copia != mv['budget'] # creo un df que me indique donde hubo cambios
df_cambios.value_counts() # verifico que haya habido cambios, los valores igual a True indican que si sucedieron cambios

# en conclusion se produjeron 3 cambios, en toda nuestra columna 'budget' teniamos 3 registros en los que
# su valor era una cadena que no se podia transformar a float

False    45379
Name: budget, dtype: int64

### 4- `Campos con fecha`: no poseen el formato correcto.
    Se debe transformar cada uno de estos campos al formato *AAAA-mm-dd*.
    Se debe crear una nueva columna `release_year` donde se extraerá el año de la fecha de estreno.

In [29]:
a = "22/04/2023"

a = pd.to_datetime(a, format='%d/%m/%Y').date()

a

datetime.date(2023, 4, 22)

In [33]:
mv['release_date'] = pd.to_datetime(mv['release_date'], format='%Y-%m-%d')

# dentro de esta columna tenemos datos que no tienen la estructura de una fecha, habra que transformarlos

ValueError: time data "1" at position 10394 doesn't match format specified

In [48]:
# analicemos que pasa en el indice que nos dice el error
mv.loc[mv['release_date']=='1'].values
# aca podemos darnos cuenta que el orden de nuestras columnas esta desfasado, fijemonos que los valores que tiene este registro
# comienzan desde el campo 'overview'.

# para hacer esta transformacion podria asumir algunos supuestos
# 1- que todos los registros que estan desfasados comienzan con el campo 'overview'
# 2- que este campo posee un string con mas de 10 caracteres, cantidad que posee una cadena con estructura de fecha

# Asumiendo estas teorias deberiamos aplicar a esta columna algun metodo para strings que pueda contar los caracteres en cada registro
# yo agregaria al lado de esta columna, otra columna que me vaya diciendo el tamaño que tiene 'release_date' (que en realidad es 'overview')

array([[' Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.',
        '1.931659', '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg',
        "[{'name': 'Aniplex', 'id': 2883}, {'name': 'GoHands', 'id': 7759}, {'name': 'BROSTA TV', 'id': 7760}, {'name': 'Mardock Scramble Production Committee', 'id': 7761}, {'name': 'Sentai Filmworks', 'id': 33751}]",
        "[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'JP', 'name': 'Japan'}]",
        '2012-09-29', '0', '68.0',
        "[{'iso_639_1': 'ja', 'name': '日本語'}]", 'Released', nan,
        'Mardock Scramble: The Third Exhaust', 'False', '7.0', '12', nan,
        nan, nan, nan, nan, nan, nan, nan, nan]], dtype=object)

### 5- `revenue`, `budget`: a partir de estos campos se debe crear una nueva columna `return`, retorno de inversión.
    Esta columna sera el resultado de `revenue`/`budget` y cuando no hay datos disponibles debera retornar *0*.

### 1- `belongs_to_collection`, `production_companies`, `genres`, `spoken_languages`: los datos estan anidados
    Se debe buscar la forma de desanidarlos, o bien, buscar la forma de que los datos se puedan consultar sin desanidarlos.

#### **belong_to_collection**

In [ ]:
# hacemos una copia de nuestra columna original para poder trabajarla
btc_copy = mv.belongs_to_collection # no es Bitcoin, es belong_to_collection

In [ ]:
# notemos que en nuestra columna 'belong_to_collection' podemos encontrar dos tipos de variables:

print(type(btc_copy[4]))
# donde tenemos los datos anidados, el tipo de dato sera string

print(type(btc_copy[3]))
# donde tenemos NaN, el tipo de dato sera float

Con esto se podria hacer una primer transformacion. La idea principal sería, que todos nuestros datos en 'belong_to_collection' sean diccionarios los cuales se pueda acceder facilmente a ellos.

------

##### **DEBO OPTIMIZAR ESTO!!!**

In [ ]:
### OPCION A: tarda mas de 20 segundos en ejecutarse


#   import ast # este modulo nos servira para transformar de string a diccionario
#   for i, registro in enumerate(btc_copy): # para cada registro en mi columna belng_to_collection
#       if (type(registro) == str):
#           btc_copy[i] = ast.literal_eval(btc_copy[i]) # transformamos str -> dict mediante funcion literal_eval
#       else: # sino, transformamos float -> dict
#           btc_copy[i] = {
#               'id':'Sin datos',
#               'name':'Sin datos',
#               'poster_path': 'Sin datos',
#               'backdrop_path': 'Sin datos'
#           }


### Resuelve el problema, pero es una locura que tarde 17 segundos

In [ ]:
### OPCION B: es 200 veces mas rapido que la OPCION A. Esto es por la funcion apply que usamos de Pandas, es mucho mas rapida
### que un bucle for tradicional
import ast

def transformar_registro_btc(registro):
    if isinstance(registro, str):
        nuevo_registro = ast.literal_eval(registro)
        nuevo_registro['id_belong_to_collection'] = nuevo_registro['id']
        del nuevo_registro['id']
        return nuevo_registro
    else:
        return {
            'id': 'Sin datos',
            'name': 'Sin datos',
            'poster_path': 'Sin datos',
            'backdrop_path': 'Sin datos'
        }

# Aplicar la función personalizada a la columna 'btc_copy'
btc_copy = btc_copy.apply(transformar_registro_btc)

# Si es necesario, convierte la columna 'btc_copy' en un DataFrame
# btc_copy = pd.DataFrame(btc_copy)

btc_copy


------

In [ ]:
print(btc_copy[4],type(btc_copy[4]))
print(btc_copy[3],type(btc_copy[3]))
# Efectivamente, logramos transformar todos los registros de esta columna a diccionarios

Ahora la pregunta sería, como podria transformar este array de diccionarios, en un Dataframe de Pandas en el que cada una de las keys de los diccionarios sea una columna??

In [ ]:
belong_to_collection_df = pd.json_normalize(btc_copy) # funcion para expandir diccionarios en columnas
# tambien me lo transforma en Dataframe

### Seria conveniente usar esta funcion para el momento en el que tenga que consultar los registros de btc_copy

Con esto, logre crear un Dataframe en el que pueda acceder a los datos de la columna 'belong_to_collection' fácilmente

In [ ]:
belong_to_collection_df[belong_to_collection_df['name']=='Toy Story Collection']

#### **production_companies**